In [2]:
import os
import sys
import h5py
import glob
import time
import numpy as np
import pandas as pd
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from skyfield.api import load, wgs84, EarthSatellite

In [3]:
### FUNCTIONS I THINK ARE NEEDED TO TRANSFORM PIXEL COORDS TO LOOK AT FOV

def world2pix(lst, ra, dec, jd=None, nx=4008, ny=2672, margin=50):
    xwcs, ywcs = world2wcs(wcspars, ra, dec, lst, jd) # Compute wcs-only pixel coordinates
    mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin) # Remove coordinates not within the margins
    xwcs, ywcs = xwcs[mask], ywcs[mask]        
    xpix, ypix = wcs2pix(polpars, xwcs, ywcs) # Convert to actual pixel coordinates.
    return xpix, ypix, mask


def clip_rectangle(x, y, nx, ny, margin):
    maskx = (x > margin) & (x < (nx - margin))
    masky = (y > margin) & (y < (ny - margin))
    return maskx & masky


def world2wcs(wcspars, ra, dec, lst=None, jd=None):
    """ Convert world coordinates to WCS-only coordinates. """    
    if jd is not None:
        ra, dec = j2000_to_equinox(ra, dec, jd)
    w = create_wcs(wcspars, lst)
    xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
    return xwcs, ywcs


def create_wcs(wcspars, lst=None):
    """ Create and astropy WCS instance from a dictionary of parameters. """  
    from astropy import wcs
    
    if lst is not None:
        ra0, dec0 = wcspars['crval']
        ha0 = ra2ha(ra0, wcspars['lst'])
        ra0 = ha2ra(ha0, lst)
        crval = np.array([ra0, dec0])   
    else:
        crval = wcspars['crval']
        
    w = wcs.WCS(naxis=2)
    w.wcs.crpix = wcspars['crpix']
    w.wcs.cdelt = wcspars['cdelt']
    w.wcs.crval = crval
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.pc = wcspars['pc']
    return w


def ra2ha(ra, lst):
    """ Convert Right Ascension to Hour Angle. """
    ha = np.mod(lst*15. - ra, 360.)
    return ha

def ha2ra(ha, lst):
    """ Convert Hour Angle to Right Ascension. """    
    ra = np.mod(lst*15. - ha, 360.)
    return ra


def j2000_to_equinox(ra, dec, jd):
    from astropy.time import Time
    from astropy.coordinates import SkyCoord, FK5
    t = Time(jd, scale='utc', format='jd') 
    t.format = 'jyear_str'
    gc = SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2000')    
    gc = gc.transform_to(FK5(equinox=t))    
    return gc.ra.value, gc.dec.value


def wcs2pix(polpars, xwcs, ywcs):
    """ Convert WCS-only coordinates to pixel coordinates. """     
    dx, dy = leg2d_eval(xwcs, ywcs, polpars['x_wcs2pix'], polpars['y_wcs2pix'], 
                        polpars['order'], polpars['nx'], polpars['ny'])    
    xpix, ypix = xwcs + dx, ywcs + dy
    return xpix, ypix


def leg2d_eval(x, y, a, b, order=6, nx=4008., ny=2672.):
    """ Evaluate 2-D polynomials. """
    mat, idx1, idx2 = leg2d_mat(x, y, order, nx, ny)
    xres = np.dot(mat, a[idx1, idx2])
    yres = np.dot(mat, b[idx1, idx2])
    return xres, yres  


def leg2d_mat(x, y, order=6, nx=4008., ny=2672.):
    """ Create a matrix for fitting 2-D polynomials of the given order. """ 
    from numpy.polynomial import legendre
    x = np.array(x)
    y = np.array(y)
    mat1 = legendre.legvander(2*(x/nx) - 1, order+1)
    mat2 = legendre.legvander(2*(y/ny) - 1, order+1)
        
    idx1, idx2 = np.indices((order+1, order+1))
    mask = ((idx1 + idx2) <= order)
    idx1, idx2 = idx1[mask], idx2[mask]        
        
    mat = mat1[:,idx1]*mat2[:,idx2]
    return mat, idx1, idx2

In [4]:
# For an image:
    # Check time of observation
    # Check if starlinks pass for that time
    # Now run the FOV test
    # If in FOV --> selection criteria checks (time, disk illumination, sat above horizon)
    # Check which camera would find the satellite ..?? Probably need to do this independently of image
    # If everything checks out --> keep image
    
    
# Problem - current way of checking FOV requires an image to be passed (so as to get the wcspars)
# Doing it this way means that the camid will have to be defined beforehand
# But this defeats the purpose of finding which camera will capture a potential starlink trail 
# Through this way, we may get a no sighting but in reality there is, just in a different camera

# Starting here - checking each camid

In [5]:
def reduce_tles(camid):

    # Load TLEs for all passages
    satfiles = '../test_data/3leComplete.txt'
    with open(satfiles) as f:
        all_tles = f.readlines()
        f.close()   

    # Split TLE list into individual lists for each TLE
    all_tles = [i.strip() for i in all_tles]
    tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

    # Reduce TLEs to Starlink only
    starlink_tles = []
    for tle in tles:
        if "STARLINK" in tle[0]:
            starlink_tles.append(tle)

    # Obtain satellite passages
    passed_sats = pd.read_pickle(f'../test_data/passages/passed_satellites_20221023{camid}.p')

    # Find any Starlink TLEs in the passages
    idx = []
    flatlist = np.asarray(starlink_tles).flatten()
    for key in passed_sats.keys():
        line1 = passed_sats[key]['TLE line1'].strip()
        i = np.where(flatlist == line1)[0] 
        if i.size > 0:
            idx.append(i[0] - 1) #appending the name of the starlink sat

    # Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
    orig_idx = [int(x/3) for x in idx]
    passed_tles = [starlink_tles[i] for i in orig_idx]

    # Remove 0 labeling of first line of TLE because that's the proper format
    for tle in passed_tles:
        tle[0] = tle[0][2:]
         
    #print(f'Number of satellites recorded for this day: {len(all_tles)}')
    #print(f'Number of them that were Starlinks: {len(starlink_tles)}')
    #print(f'Number of Starlinks that passed MASCARA: {len(passed_tles)}')
    
    return passed_tles

In [6]:
# starlinks = reduce_tles('LSC')

In [7]:
def image_data(camid):
    
    images = glob.glob(f'../test_data/diff_images/{camid}/diff_*.fits.gz')

    imgdata = {}
    for img in images:
        header = pf.getheader(img) 
        lstseq = img[-19:-11]
        midlst = header['MIDLST']
        midJD = header['MIDJD']
        nx = header['XSIZE']    
        ny = header['YSIZE']

        imgdata[lstseq] = {}
        imgdata[lstseq]['midLST'] = midlst
        imgdata[lstseq]['midJD']  = midJD
        imgdata[lstseq]['nx'] = nx
        imgdata[lstseq]['ny'] = ny
    
    return imgdata

In [8]:
# imgdata = image_data('LSC')

In [9]:
def passage_check(midJD, tles, passages):
        
    idx_reduced = []
    sats = passages[midJD].keys()
    
    satnums = []
    for tle in tles:
        satnums.append(tle[1].split()[1])

    # Cross-reference
    for i, sat in enumerate(sats):
        if sat in satnums:
            idx = satnums.index(sat)
            idx_reduced.append(idx)
  
    return idx_reduced

In [10]:
def get_siteinfo():
    confdir = '../fotos-python3/bringfiles/siteinfo.dat'
    dtype = [('sitename', '|U20'), ('lat', 'float32'), ('lon', 'float32'), ('height', 'float32'), ('ID', '|U2')]
    siteinfo = np.genfromtxt(confdir, dtype=dtype)   
    mask = siteinfo['ID'] == 'LS'
    siteinfo = siteinfo[mask]
    return siteinfo

In [11]:
### Get timerange of all images

def image_timerange(imgdata):
    
    dates = []
    for lst in list(imgdata):
        dates.append(imgdata[lst]['midJD'])
    
    oldest = min(dates)
    newest = max(dates)

    # Converting to utc for convenience
    ts = load.timescale()
    newest = ts.tt_jd(newest).utc_strftime()
    oldest = ts.tt_jd(oldest).utc_strftime()

    beg = pd.to_datetime(oldest)
    end = pd.to_datetime(newest)
    rng = pd.date_range(beg, end, freq='0.05H').to_pydatetime().tolist() #every 3 minutes
    timerange = ts.from_datetimes(rng)
    
    return timerange

In [12]:
def check_illumination(imgdata, midJD, timerange, sat):

    # Check when satellite is illuminated 
    timerange = image_timerange(imgdata)
    sunlit = sat.at(timerange).is_sunlit(eph)

    # Obtain the indices of the first and last True element for each sequence of True elements 
    sunlit_idx = []
    start_idx = None
    for i, elem in enumerate(sunlit):
        if elem:
            # Checking if the start of a sequence of True elements
            if start_idx is None:
                start_idx = i
        else:
            # Checking if a sequence is just ended
            if start_idx is not None:
                sunlit_idx.append((start_idx, i-1)) # -1 because we're at the first False element after True sequence
                start_idx = None

    # If the last element is True, we need to append its index as well
    if start_idx is not None:
        sunlit_idx.append((start_idx, len(sunlit)-1))
    
    # Getting the times from the indices
    sunlit_times = []
    for idx in sunlit_idx:
        sunlit_times.append([timerange.tt[idx[0]], timerange.tt[idx[1]]])

    # Check if midJD is within each period and flag it if so
    illuminated = False
    for sunlitrng in sunlit_times:
        if midJD >= (sunlitrng[0] - 1/86400) and midJD <= (sunlitrng[1] + 1/86400): #adding a bumper of second
            illuminated = True
            if illuminated:
                break

    return illuminated

In [16]:
### For Jupyter Notebook playaround, will use LSC and LSE only

### THERE ARE A NUMBER OF THINGS WE DON'T WANT TO REPEAT IN THE MAIN LOOP

# DE421 planetary ephemeris (to get position of Sun)
eph = load('de421.bsp')

def check_each_camid():
    
    print('Getting Image Data')
    LSC = image_data("LSC")
    LSE = image_data("LSE")
    
    print('Getting Timeranges')
    trC = image_timerange(LSC)
    trE = image_timerange(LSE)

    print('Getting Fast Curves')
    fastC = h5py.File("../fast_20221023LSC.hdf5", "r")    
    fastE = h5py.File("../fast_20221023LSE.hdf5", "r")    

    print('Getting Passages')
    psgsC = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")
    psgsE = pd.read_pickle("../test_data/passages/passages_20221023LSE.p")

    print('Getting TLEs')
    tlesC = reduce_tles("LSC")
    tlesE = reduce_tles("LSE")
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
    
    # Want to check the date for each camid before moving on to the next date
    # The LST sequences will be the same for each camid
    
    print('Beginning Loop \n')
    
    result = []
    for seq in fastC['station']['lstseq'][:]: 
        lstseq = str(seq)

        for data, timerange, camid, fast, starlinks, passages in zip(
            [LSC, LSE], [trC, trE], ['LSC', 'LSE'], [fastC, fastE], [tlesC, tlesE], [psgsC, psgsE]):

            if lstseq not in data.keys():
                #print(f'{lstseq} not in directory')
                continue #this is only needed if not going through ALL images!

            # Set time of image
            midJD = data[lstseq]['midJD']
            ts = load.timescale()
            t  = ts.tt_jd(midJD)

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            else:
                print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
                # Attaining the astrometric solution (depends on FAST file and LSTseq)
                astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                lst=fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                nx = data[lstseq]['nx']
                ny = data[lstseq]['ny']

                wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                            'crpix' : fast['astrometry/crpix'][astro].copy(),
                            'cdelt' : [0.02148591731740587,0.02148591731740587],
                            'pc'    : fast['astrometry/pc'][astro].copy(),
                            'lst'   : lst }

                polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                            'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                            'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                            'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                            'nx'    : nx,
                            'ny'    : ny,
                            'order' : order }

                midLST = data[lstseq]['midLST']

                for idx in idx_reduced:
                    line1 = starlinks[idx][0]
                    line2 = starlinks[idx][1]
                    line3 = starlinks[idx][2] 
                    sat = EarthSatellite(line2, line3, line1, ts)

                    diff = sat - mascara
                    topocentric = diff.at(t)
                    alt, az, dist = topocentric.altaz()
                    
                    # Criteria check: if satellite is >20 degrees above the horizon
                    if alt.degrees >= 20:
                        
                        # Criteria check: if satellite is illuminated
                        illuminated = check_illumination(data, midJD, timerange, sat)
                        
                        if illuminated:
                            ra, dec, radec_dist = topocentric.radec() 
                            radeg = ra._degrees
                            dedeg = dec._degrees
                            ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                            w = create_wcs(wcspars, midLST)
                            xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                            mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)

                            if mask:
                                print(f"At least one Starlink should be in {camid} FOV")
                                image = lstseq + camid
                                result.append(image)
                                break #so to continue to next LSTSEQ if at least one starlink is found!
                
                print('\n')



    fastC.close()
    fastE.close()
    
    if len(result) > 0:
        np.savetxt('../selection_pool.txt', result, fmt='%s')
        
    return result

In [17]:
result = check_each_camid()

Getting Image Data
Getting Timeranges
Getting Fast Curves
Getting Passages
Getting TLEs
Beginning Loop 

There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263


There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377
At least one Starlink should be in LSE FOV


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907


There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687


There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045


There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980


There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733


There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920


There are 1 Starlinks passing overhead LSC for LSTSEQ=48510957




# Now we want to incorporate the selection criteria

## Check disk illumination and if satellite above horizon

So, if a satellite is in the FOV --> check if above horizon

Then we'll deal with disk illumination by using the Sunlit brute force method

In [17]:
# # Lets collect the sat number for each JD in the passages file
# passages_satnums = []
# for JD in JDs:
#     passages_satnums.extend(list(passages[JD]))
    
# # Now remove duplicates
# passages_satnums = list(set(passages_satnums))

# print(len(satnums))
# print(len(passages_satnums))

# if set(passages_satnums) == set(satnums):
#     print("They have the same elements")
# else:
#     print("They do not have the same elements")

## Test Pyephem vs Skyfield

### Check if satellite coords in passages match those calculated 

In [16]:
passed = pd.read_pickle('../test_data/passages/passed_satellites_20221023LSC.p')
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")

tles = reduce_tles('LSC')
jds = list(passages)

In [45]:
satnums = []
for tle in tles:
    satnums.append(tle[1][2:8])

stop = False
for i, jd in enumerate(jds):
    sats = list(passages[jd])
    for j, satnum in enumerate(satnums):
        if satnum in sats:
            print(sats.index(satnum))
            print(i)
            print(j)
            stop = True
            break
    if stop:
        break

142
0
3


In [46]:
tles[3]

['STARLINK-2606',
 '1 48386U 21038AK  22295.51609890  .00000159  00000-0  29547-4 0  9997',
 '2 48386  53.0552 127.0618 0001573 106.4026 253.7136 15.06396948 81211']

In [47]:
testsat = sats[142]
testJD  = jds[0]
print(testsat)
passages[testJD][testsat]

48386U


{'start': {'jd': 2459876.516196968,
  'lst': 21.832494040233648,
  'ra': 318.5476590312142,
  'dec': -48.97685923149278,
  'x0': 2378.002001747682,
  'y0': 2162.5718866860293},
 'end': {'jd': 2459876.5162708443,
  'lst': 21.834267075817095,
  'ra': 322.9977361234362,
  'dec': -52.35211620986041,
  'x0': 2233.8918844758578,
  'y0': 2332.574848495738}}

In [48]:
print('From passages file:\n')
print(passages[testJD][testsat]['start']['ra'])
print(passages[testJD][testsat]['start']['dec'])

From passages file:

318.5476590312142
-48.97685923149278


In [49]:
ts = load.timescale()
line1 = tles[3][0]
line2 = tles[3][1]
line3 = tles[3][2]
sat = EarthSatellite(line2, line3, line1, ts)

site = get_siteinfo()
mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
diff = sat - mascara
topocentric = diff.at(ts.tt_jd(testJD))
alt, az, dist = topocentric.altaz()
ra, dec, radec_dist = topocentric.radec() 

print(ra._degrees, dec._degrees)

ra_test, dec_test = j2000_to_equinox(ra._degrees, dec._degrees, testJD)
print(ra_test, dec_test)

292.23936732678953 -10.878966819859329
292.55413739978684 -10.83058212781927


### Compare (ra, dec) computation

Need to select an image (i.e. an LSTSEQ) with a Starlink in it so that I can compare locations of the satellites.

Could also do for a non Starlink!

In [26]:
import ephem

In [73]:
fast = h5py.File("../fast_20221023LSC.hdf5", "r")
lstseq = '48506377'
astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
lst=fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
nx = imgdata[lstseq]['nx']
ny = imgdata[lstseq]['ny']

wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
            'crpix' : fast['astrometry/crpix'][astro].copy(),
            'cdelt' : [0.02148591731740587,0.02148591731740587],
            'pc'    : fast['astrometry/pc'][astro].copy(),
            'lst'   : lst }

polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
            'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
            'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
            'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
            'nx'    : nx,
            'ny'    : ny,
            'order' : order }

### So, the (ra, dec) for the satellite differs between the computed skyfield one, and the one in the passages

Are we sure we are looking at the correct JD? Is it midJD in both cases?

I'm computing (ra,dec) using the midJD as the time of observation.

I think that, in the passages file, the keys are the JD for the observation. But this doesn't correspond to the midJD of the image..

DOESN'T MATTER!

In [166]:
imgdata = image_data('LSC')
starlinks = reduce_tles('LSC')
midJD = imgdata[list(imgdata)[0]]['midJD']
midLST = imgdata[list(imgdata)[0]]['midLST']

line1 = starlinks[24][0]
line2 = starlinks[24][1]
line3 = starlinks[24][2] 

# Skyfield
ts = load.timescale()
sat_sky = EarthSatellite(line2, line3, line1, ts)
mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
diff = sat_sky - mascara
topocentric = diff.at(ts.tt_jd(midJD))
ra, dec, radec_dist = topocentric.radec() 
ra_sky = ra._degrees
de_sky = dec._degrees

# Pyephem
sat_eph = ephem.readtle(line1, line2, line3)
site = get_siteinfo()
obs = ephem.Observer()
obs.lat = site['lat']*np.pi/180
obs.long = site['lon']*np.pi/180
obs.elev = site['height']
# edate = ephem.Date('2000/01/01 12:00:00.0')
edate = ephem.Date('2022/10/23 12:00:00.0')
obs.epoch = edate  
exptime = 6.382928100414574 #exposure time (making this up!)

#pyephem.date is the number of days since 1899 December 31 12:00 UT (2415020 JD) 
obs.date = midJD - 2415020-0.5*exptime/86400
sat_eph.compute(obs)

ra_eph = sat_eph.a_ra * (180/np.pi)
de_eph = sat_eph.a_dec * (180/np.pi)


print('Skyfield:')
print(f'(Ra, Dec) = {ra_sky}, {de_sky}\n')

print('Pyephem:')
print(f'(Ra, Dec) = {ra_eph}, {de_eph}')

Skyfield:
(Ra, Dec) = 267.72426773817983, 49.10208090218334

Pyephem:
(Ra, Dec) = 272.5392700993798, 47.26397566836103


In [105]:
# .fraction_illuminated(eph['sun'])

In [135]:
# Angle between the sun and the observer on earth as observed by a satellite: 

ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']
sun = eph['sun']

test = earth + sat_sky
t = ts.tt_jd(midJD)

sun_position = test.at(t).observe(sun)
earth_position = test.at(t).observe(earth+mascara)
x
phase_angle = sun_position.separation_from(earth_position)

print(phase_angle.degrees)

32.95338437287889


In [120]:
# Angle between the observer and the Sun from the satellite's point of view:

obs  = earth + mascara
test = earth + sat_sky

topo_sat_obs = test.at(t).observe(obs).apparent()
topo_sat_sun = test.at(t).observe(sun).apparent()

phase_angle = topo_sat_obs.separation_from(topo_sat_sun)
print(phase_angle.degrees)

32.95327763516486


In [133]:
test.at(t).phase_angle(eph['sun']).degrees #the position’s phase angle: the angle of Sun-target-observer

0.30210702901977293

In [130]:
test.at(t).fraction_illuminated(eph['sun'])

0.9999930495104037

In [145]:
passages[midJD]

{'00271U': {'start': {'jd': 2459876.524396782,
   'lst': 22.029828387681224,
   'ra': 332.33300153467945,
   'dec': -9.98380860759643,
   'x0': 1868.005478726076,
   'y0': 246.65119609268362},
  'end': {'jd': 2459876.524470658,
   'lst': 22.031601416820177,
   'ra': 332.3429390489783,
   'dec': -10.794675019528274,
   'x0': 1870.7090208664158,
   'y0': 290.526765603692}},
 '02359U': {'start': {'jd': 2459876.524396782,
   'lst': 22.029828387681224,
   'ra': 343.51084143001225,
   'dec': -37.50302750000799,
   'x0': 1544.1158669937806,
   'y0': 1659.6099017598217},
  'end': {'jd': 2459876.524470658,
   'lst': 22.031601416820177,
   'ra': 343.5532183917439,
   'dec': -38.01296684483998,
   'x0': 1546.9874307546002,
   'y0': 1684.3058464784028}},
 '02766U': {'start': {'jd': 2459876.524396782,
   'lst': 22.029828387681224,
   'ra': 349.4041586259797,
   'dec': -19.054148008148037,
   'x0': 1100.857260965852,
   'y0': 814.2596789988751},
  'end': {'jd': 2459876.524470658,
   'lst': 22.031601

In [156]:
site = get_siteinfo()
mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
diff = sat_sky - mascara
topocentric = diff.at(ts.tt_jd(JDs[0] - 3.2/86400))
ra, dec, radec_dist = topocentric.radec() 

print(ra._degrees)
print(dec._degrees)


198.66682909175736
-31.76723470552273


In [157]:
topocentric = diff.at(ts.tt_jd(JDs[0]))
ra, dec, radec_dist = topocentric.radec() 

print(ra._degrees)
print(dec._degrees)

198.79038361657393
-31.864258414056405


In [152]:
JDs[0]

2459876.516233906

In [151]:
print('From passages file:\n')
print(passages[JDs[0]]['48386U']['start']['ra'])
print(passages[JDs[0]]['48386U']['start']['dec']) 

From passages file:

318.5476590312142
-48.97685923149278
